# Integrated Hybrid Computing: Blending classical and quantum computing
In this sample, you will discover how to blend classical and quantum instructions in the same program, all fully processed by the quantum computing backend.
You can learn more about integrated hybrid computing at https://aka.ms/AQ/Hybrid/Docs

## Connect to the Azure Quantum workspace
First, find the resource ID of your Azure Quantum workspace. You can copy/paste this from the top-right corner of your Quantum Workspace page in Azure Portal.

Next, you can run `%azure.connect` to connect to the workspace and display the list of provisioned targets that support execution of Q# programs. If you are prompted to login, be sure to use the same account you used to create your Azure Quantum workspace.

In [ ]:
%azure.connect "/subscriptions/subscriptionId/resourceGroups/resourceGroupName/providers/Microsoft.Quantum/Workspaces/workspaceName" location="westus"

## Define the CheckGHZ operation

This program will examine a 3 qubit Greenberger–Horne–Zeilinger (GHZ) state and count the number of times it sees the entanglement fail out of 10 attempts. Unlike in perfect, noiseless simulations, entanglement failures can occur in real-world runs, which are subject to the effects of noise.

In the CheckGHZ() operation, you see three capabilities of integrated Hybrid being demonstrated:
1. **Branching**: based on measurement results you can adjust the classical code being performed
```Q#
   if not (r0 == r1 and r1 == r2)
```
2. **Classical compute**: here an arithmetic operation is performed
```Q#
   set mismatch += 1;
```
3. **While the physical qubits are still coherent**, #1 and #2 are performed 


Please note that you will see a warning displayed (QS5026). It can be ignored. It will be addressed in a subsequent release of the Quantum Development Kit.

In [ ]:
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Convert;

operation CheckGHZ() : Int {
    use q = Qubit[3];
    mutable mismatch = 0;
    for _ in 1..10 {
        H(q[0]);
        CNOT(q[0], q[1]);
        CNOT(q[1], q[2]);

        // Measures and resets the 3 qubits
        let (r0, r1, r2) = (MResetZ(q[0]), MResetZ(q[1]), MResetZ(q[2]));

        // Adjusts classical value based on measurement results
        if not (r0 == r1 and r1 == r2) {
            set mismatch += 1;
        }
    }
    return mismatch;
}

## Submit the operation
Using 50 shots, as configured below, this program will use approximately 10.65 units (HQCs or eHQCs).<br/>
If the job has not been completed after the client times out (300 seconds), you can query the status using `%azure.status` followed by querying the output using `%azure.output`.<br/>
If the kernel was restarted after the program had been submitted, you will have to use `%azure.status <job_id>` to get the status of the job and `%azure.output <job_id>` to retrieve the results.<br/>
You can also check the status of the job under the **Job management** section of the portal.

**Understanding the results**<br/>
The histogram reports the ratio, out of the number of shots performed, that a given number of entanglements failed.<br/>
The histogram values can range from 0 (no entanglement failed) to 10 (all entanglements failed).

In [ ]:
%azure.target quantinuum.sim.h1-1e-preview
%azure.target-capability AdaptiveExecution
%azure.execute CheckGHZ shots=50 timeout=300 poll=10

In [ ]:
%azure.status

In [ ]:
%azure.output

## ↗ Learning more about Integrated hybrid
Now that you have run this sample, you can further learn and experiment with the _Error Correction_ and the _Iterative Phase Estimation_ samples also available in the gallery.

To learn more, please refer to the [online documentation](https://aka.ms/AQ/Hybrid/Docs).